<a href="https://colab.research.google.com/github/bogus1aw/text-classification-benchmark/blob/main/M_herBERT_PolEmo2_0_raw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HerBERT benchmark for PolEmo2.0 raw dataset 

In [ ]:
# check available GPU
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Tesla T4, 460.32.03, 15109 MiB


In [ ]:
%%capture
!pip install datasets transformers

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
cuda.is_available()

True

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import datetime
import time
import math
import shutil

In [ ]:
def load_corpora_to_dataframe(corpora):
  data = open(corpora).read()
  labels, texts = [], []
  for i, line in enumerate(data.split("\n")):
      content = line.split()
      if len(content) > 0: 
        labels.append(content[-1])
        texts.append(" ".join(content[:-1]))
  # return texts, labels
  # create a dataframe using texts and labels
  trainDF = pd.DataFrame()
  trainDF['texts'] = texts
  trainDF['labels'] = labels
  return trainDF


# 1. load corpora
# 2. create training, test fractions
# 3. create specific No. per class fractions
# 4. create train, val sets
# 5. create new fresh model (tokenizer can stay the same)
# 6. tokenize and encode train, test 


In [ ]:
import torch
# from transformers import AutoTokenizer, RobertaForSequenceClassification, EvalPrediction
from transformers import HerbertTokenizer, RobertaForSequenceClassification, EvalPrediction


# tokenizer = AutoTokenizer.from_pretrained("allegro/herbert-base-cased")
# model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=4)
tokenizer = HerbertTokenizer.from_pretrained("allegro/herbert-klej-cased-tokenizer-v1")
model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=4)

Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream

In [ ]:
from sklearn import preprocessing
encoder = preprocessing.LabelEncoder()

def build_databases(train_data, dev_data, test_data):

  max_length = 200
  train_encodings = tokenizer(train_data['texts'].to_list(), truncation=True, padding=True, max_length=max_length)
  val_encodings = tokenizer(dev_data['texts'].to_list(), truncation=True, padding=True, max_length=max_length)
  test_encodings = tokenizer(test_data['texts'].to_list(), truncation=True, padding=True, max_length=max_length)

  # encode labels
  train_labels = encoder.fit_transform(train_data['labels'].to_list())
  val_labels = encoder.fit_transform(dev_data['labels'].to_list())
  test_labels = encoder.fit_transform(test_data['labels'].to_list())

  # build pyTorch dataset
  import torch

  class wikiDataset(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
          item['labels'] = torch.tensor(self.labels[idx])
          return item

      def __len__(self):
          return len(self.labels)

  train_dataset = wikiDataset(train_encodings, train_labels)
  val_dataset = wikiDataset(val_encodings, val_labels)
  test_dataset = wikiDataset(test_encodings, test_labels)
  return train_dataset, val_dataset, test_dataset


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          
    evaluation_strategy = "epoch",
    num_train_epochs=4,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=100,                
    weight_decay=0.01,               
    logging_dir='./logs',            
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)

from datasets import load_metric
import numpy as np
metric = load_metric('accuracy')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    # print(predictions[:10])
    # print(labels[:10])
    return metric.compute(predictions=predictions, references=labels)

def get_trainer(model, train_dataset, val_dataset):
  trainer = Trainer(
      model=model,                         # the instantiated 🤗 Transformers model to be trained
      args=training_args,                  # training arguments, defined above
      train_dataset=train_dataset,         # training dataset
      eval_dataset=val_dataset,             # evaluation dataset
      tokenizer=tokenizer,
      compute_metrics=compute_metrics
  )
  return trainer


In [ ]:
def write_to_logs(values):
  with open(metrice_path, 'a') as f:
    f.write(values)

In [ ]:
timestamp = datetime.datetime.now().replace(microsecond=0).isoformat().replace(':', '-')
metrice_path = '/content/drive/MyDrive/metrics/hebert_PolEmo2.0_raw2.0' + timestamp + '.txt'
fig_path = '/content/drive/MyDrive/figures/'
dataset_path = '/content/drive/MyDrive/master_datasets/dataset_conll/'


no_samples_per_class = [1, 3, 5, 8, 10, 20, 30, 60, 100, 200] ###### FINALL SAMPLES LIST  
repetitions = 5

domains = [
           ('all', 'MDT-A'),
           ('hotels', 'SDT-H'),
           ('medicine', 'SDT-M'),
           ('products', 'SDT-P'),
           ('reviews', 'SDT-R')
           ]


In [ ]:
def benchmark(train, dev, test, n_sample, domain):
  accuraccy, train_time, eval_time = [], [], []
  for repeat in range(repetitions):
    # print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    # print('training for n_sample = ', n_sample)
    # print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    # model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-base-cased", num_labels=4) ######## Change num of labels !!!!!
    model = RobertaForSequenceClassification.from_pretrained("allegro/herbert-klej-cased-v1", num_labels=4) ######## Change num of labels !!!!!
    train_fraction = train.groupby(['labels']).sample(n=max(math.floor(n_sample * 0.9), 1), replace=True)
    dev_fraction = dev.groupby(['labels']).sample(n=max(math.floor(n_sample*0.1), 1), replace=True)
    train_dataset, val_dataset, test_dataset = build_databases(train_fraction, dev_fraction, test)
    trainer = get_trainer(model=model, train_dataset=train_dataset, val_dataset=val_dataset)
    
    train_time_start = time.time()
    trainer.train()
    train_elapsed_time = time.time() - train_time_start
    train_time.append(train_elapsed_time)
    # trainer.evaluate()
    metrics = trainer.predict(test_dataset)
    accuraccy.append(metrics.metrics['eval_accuracy']) 
    eval_time.append(metrics.metrics['eval_runtime']) 
    to_save = f'domiain {domain} n_samples_per_class={n_sample}, repeat={repeat}, time_elapsed={train_elapsed_time}, {metrics.metrics}\n' 
    print(to_save)
    write_to_logs(to_save)
    shutil.rmtree('./results') # deleate checkpoints files

  return accuraccy, train_time, eval_time

In [ ]:
print(metrice_path)

results_a = pd.DataFrame()
results_t = pd.DataFrame()
results_e = pd.DataFrame()

for domain, ix_name in domains:
  write_to_logs('%%%%%%%%%%%%%%%% domain: ' + domain)
  CORPORA_TRAIN = dataset_path + domain + '.text.train.txt'
  CORPORA_DEV = dataset_path + domain + '.text.dev.txt'
  CORPORA_TEST = dataset_path + domain + '.text.test.txt'

  train = load_corpora_to_dataframe(CORPORA_TRAIN)
  dev = load_corpora_to_dataframe(CORPORA_DEV)
  test = load_corpora_to_dataframe(CORPORA_TEST)

  accu_list, train_t, eval_t = [], [], []

  for n_sample in no_samples_per_class:
    accuraccy, train_time, eval_time = benchmark(train, dev, test, n_sample, domain)
    accu_list.append(np.mean(accuraccy))
    train_t.append(np.mean(train_time))
    eval_t.append(np.mean(eval_time))
  
  results_a.append(pd.DataFrame(accu_list, index=no_samples_per_class, columns=[ix_name + '_R_']).T)
  results_t.append(pd.DataFrame(train_t, index=no_samples_per_class, columns=[ix_name + '_R_']).T)
  results_e.append(pd.DataFrame(eval_t, index=no_samples_per_class, columns=[ix_name + '_R_']).T)
  
display(results_a)
display(results_t)
display(results_e)

 

/content/drive/MyDrive/metrics/hebert_PolEmo2.0_raw2.02021-02-17T21-24-28.txt


Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,No log,1.474730,0.250000,0.059400,67.329000
2,No log,1.473970,0.250000,0.054200,73.866000
3,No log,1.472673,0.250000,0.044700,89.478000
4,No log,1.470563,0.250000,0.047300,84.637000


domiain all n_samples_per_class=1, repeat=0, time_elapsed=116.9348087310791, {'eval_loss': 1.3309404850006104, 'eval_accuracy': 0.41097560975609754, 'eval_runtime': 9.6005, 'eval_samples_per_second': 85.412}



Some weights of the model checkpoint at allegro/herbert-klej-cased-v1 were not used when initializing RobertaForSequenceClassification: ['pooler.dense.weight', 'pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at allegro/herbert-klej-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream

...

Epoch,Training Loss,Validation Loss,Accuracy,Runtime,Samples Per Second
1,1.127300,1.043343,0.433333,0.815800,73.545000
2,0.588700,0.454654,0.716667,0.791300,75.822000
3,0.240900,1.053047,0.766667,0.802300,74.785000
4,0.061500,0.273802,0.950000,0.821700,73.023000


domiain reviews n_samples_per_class=200, repeat=4, time_elapsed=142.56699228286743, {'eval_loss': 0.8234228491783142, 'eval_accuracy': 0.76, 'eval_runtime': 0.6641, 'eval_samples_per_second': 75.286}



""


""


""
